# [Session 0] System setup

## General information

This course is designed such that during each of the sessions participants should read the materials presented in a notebook (like this exact one) and simultaneously, whenever prompted, execute provided scripts in a UNIX shell. I use Jupyter technology here for pure documentation, these notebooks are not intented to be interactive nor executable. For their Markdown support and syntax highlighting these documents are a very convenient solution to prepare a code-based tutorial. In a few places I might present an executed bash command (through a bash kernel for Jupyter) but users should always follow instructions in a separate command line.

Thus, an internet browser is essential to follow the course materials, two options are available:
* `[local]` clone the git repository and open rendered notebooks in HTML format in your browser
* `[remote]` do not clone the repository, navigate through the GitHub and browse notebooks in JSON format (ipynb extension)

We will assume that users work on a Linux system, specifically the popular Ubuntu distribution.  
Please make sure you have the following software installed and available:
* _make_
* _cmake_
* _wget_
* _g++_
* _r-base_
* _python3 + pip_
* _vim_

In case any of those is missing in your system you may easily install them with the `apt` package manager:
```
apt install -y make cmake wget g++ r-base python3-pip vim
```

Please test these tools with the following commands:
```
make --version
cmake --version
wget --version
g++ --version
R --version
python3 --version
vim --version
```

## Docker container

For users who do not operate on Ubuntu machines we have prepared a dedicated Docker image with all the prerequisites already in place.  
Those who already set up their system as in the previous description may skip this section.

Docker Engine itself becomes the one and only dependency.  
Please go through the [official Docker documentation](https://docs.docker.com/engine/install/) for instructions on how to set it up on your operating system.

In order to pull the image prepared for this course one should execute:
```
docker pull angrymaciek/efcomp:latest
```

That command needs to be executed only once.  
Having the image available locally we may now build Docker containers based on it.

Start the container with:
```
docker run --name efcomp -it efcomp:latest
```

Please note that the container environment is temporary; after exiting it one should delete the old container before starting a new one:
```
docker rm efcomp
```

### Docker image design

This may come as bonus subsection but let us break down the _Dockerfile_ of the repository, which is a recipe for the above mentioned Docker image. This is a course in scientific computing, after all.

The first section specifies the base image for this recipe, in our case Ubuntu LTS edition.  
This is followed by a few lines of my own metadata specification for the image (`LABEL` keyword).  
Finally, we set two important environmental variables inside the container:
* `LANG=C.UTF-8` : This variable defines the default locale for the container. In this case, it is set to use the UTF-8 character encoding, which is a widely used character encoding for internationalization and multilingual support
* `DEBIAN_FRONTEND=noninteractive` : This variable is used by many package managers (such as `apt`) to control their interaction mode. By setting it to noninteractive, you are telling the package manager not to prompt for any user input during package installations or upgrades. This is useful in automated build scenarios where there may not be a terminal for interactive input (as the case is for a Docker image build process)

```dockerfile
##### BASE IMAGE #####
FROM ubuntu:22.04

##### METADATA #####
LABEL base.image="ubuntu:22.04"
LABEL version="1.1.0"
LABEL software="efcomp"
LABEL software.description="Efficient scientific computing in Python/R/C++"
LABEL software.website="https://github.com/AngryMaciek/efcomp"
LABEL software.license="GPL-3.0"
LABEL software.tags="C++, Python, R, scientific-computing"
LABEL maintainer="Maciek Bak"
LABEL maintainer.email="wsciekly.maciek@gmail.com"

##### ENVIROMENTAL VARIABLES #####
ENV LANG C.UTF-8
ENV DEBIAN_FRONTEND noninteractive

```

In the next secion we install all previously listed dependencies. The official Docker image of _Ubuntu 22.04_ do not come with these preinstalled therefore we shall use `apt` command to include them into our course-dedicated image. Upon completed installation we create a symbolic link to the Python interpreter (`python` points to the new `python3`) and clean up all the temporary files created during the process.

```dockerfile
##### SYSTEM PACKAGES #####
RUN apt update \
  && apt install -y make cmake wget g++ r-base python3-pip \
  && update-alternatives --install /usr/bin/python python /usr/bin/python3 1 \
  && apt autoremove -y \
  && apt clean -y \
  && rm -rf /var/lib/apt/lists/* /tmp/* /var/tmp/*
```

The last part of the _Dockerfile_ copies the necessary course materials into the Docker image, sets a working directory inside and specifies the the default command and/or parameters that will be executed when a container based on the Docker image is run.

```dockerfile
##### COPY THE LEARNING MATERIALS #####
COPY sessions /root/sessions
WORKDIR /root/sessions

CMD ["/bin/bash"]

```

A complete list of all the _Dockerfile_ keywords is available at:  
https://docs.docker.com/engine/reference/builder/

## GitHub Codespaces

If both of the suggested methods fail for you (i.e. no Linux machine available, Docker set up fails) there is one more alternative to try out, provided you have a GitHub account. We may "hack" the built-in _Codespaces_ mechanism and use it not for a development environment but for a cloud-based execution container. Read more about GitHub Codespaces under [this link](https://github.com/features/codespaces).

I have prepared a codespace recipe for this repository to start in the same container as described above.  

In order to start the environment please navigate to my repository page and then click the green button `Code`, followed by `Create codespace on master`. A new interactive development environment should start with a Visual Studio code IDE. The IDE part is not as relevant as the fact that we have an interactive bash shell running inside the project-specific container.

## Final words

All in all, this short session was dedicated to setting up desired workspace for the whole course.  
Regardless whether that would be on your own operating system or inside a Docker container.  
**Please make sure you are prepared before progressing any further.**

---